# Session 3: Types, type inference and stability

### OBJECTIVE: Demonstrate the dynamic programming features of Julia.

#### KR1: Benchmarked at least two(2) different implementation of the same function or process (e.g. raising each element of an array to some power `p`, random array may be used) that utilizes some parameter that can be considered a constant or declared globally. Typical methods: (1) Global variable, (2) Constant global variable, and (3) Named parameter variable.

In [1]:
using BenchmarkTools

In [5]:
p = 2

function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y ^ p
    end
    return s
end

pow_array (generic function with 1 method)

In [7]:
t = rand(100000)
@btime pow_array($t)

  5.755 ms (300000 allocations: 4.58 MiB)


33458.62132401472

In [8]:
@code_warntype pow_array(t)

Variables
  #self#::Core.Const(pow_array)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64

Body::Any
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [9]:
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [10]:
@btime pow_array2($t)

  111.183 μs (0 allocations: 0 bytes)


33458.62132401472

In [11]:
@code_warntype pow_array2(t)

Variables
  #self#::Core.Const(pow_array2)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  y::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [12]:
function pow_array3(x::Vector{Float64})
    return pow_array_inner(x, p)
end

function pow_array_inner(x, pow)
    s = 0.0
    for y in x
        s = s + y ^ pow
    end
    return s
end

pow_array_inner (generic function with 1 method)

In [13]:
@btime pow_array3($t)

  111.351 μs (1 allocation: 16 bytes)


33458.62132401472

#### KR2: Replicated the naive implementation of the polynomial in the textbook.

In [2]:
function poly_naive(x, a...)
    p=zero(x)
    for i = 1:length(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

poly_naive (generic function with 1 method)

In [3]:
f_naive(x) = poly_naive(x, 1, 2, 3, 4, 5, 6, 7, 8, 9)

f_naive (generic function with 1 method)

In [28]:
x = 3.5
bench_naive = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 732 evaluations.
 Range (min … max):  171.801 ns …  2.514 μs  ┊ GC (min … max): 0.00% … 91.42%
 Time  (median):     174.268 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   182.848 ns ± 53.829 ns  ┊ GC (mean ± σ):  0.61% ±  2.05%

  ▆█▄▄▁      ▁               ▂▂▁ ▁                             ▁
  █████▆▃▅▃▅███▇▇▇▇▇▇▆▇▇███▇██████▇█▆█▇▇▇▇▆▇▆▆▆▆▆▆▆▅▅▅▃▄▅▅▃▄▄▅ █
  172 ns        Histogram: log(frequency) by time       257 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

#### KR3: Replicated the naive implementation of the Horner’s method for the same polynomial.

In [14]:
function poly_horner(x, a...)
    b=zero(x)
    for i = length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

poly_horner (generic function with 1 method)

In [27]:
f_horner(x) = poly_horner(x, 1,2,3,4,5,6,7,8,9)
bench_horner = @benchmark f_horner($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.886 ns … 135.951 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.291 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.861 ns ±   5.527 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆       ▃▆▄                                                ▁
  ██▇▇███▇▇████▅▄▅▅▄▄▄▅▃▆▁▃▄▆▅▅▅▅▅▅▆▆▇▆▆▆▅▅▅▆▅▆▅▅▅▅▄▃▃▅▅▅▃▄▅▅ █
  4.89 ns      Histogram: log(frequency) by time      29.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

#### KR4: Replicated the macro implementation of the Horner’s method of the same polynomial.

In [17]:
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

In [18]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [26]:
bench_macro = @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.046 ns … 8.646 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.049 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.052 ns ± 0.086 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃ █ █ ▇ ▆ ▄ ▃ ▅ ▆ ▅  ▁   ▂ ▄ ▄                    ▁ ▁ ▁   ▂
  █▁█▁█▁█▁█▁█▁█▁█▁█▁█▁▁█▁█▁█▁█▁█▁█▁▇▁█▁▇▁▁▇▁█▁▇▁▇▁▇▁█▁█▁█▁▇ █
  0.046 ns    Histogram: log(frequency) by time    0.073 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

#### KR5: Table showing how many minutes will the function evaluations in both KR3 and KR4 be reduced if KR2 requires 24hours of runtime.

In [44]:
method_names = ["Naive", "Horner", "Macro"]
speedup = [median(i.times) for i in [bench_naive, bench_horner, bench_macro]] / median(bench_naive.times)
minutes = speedup * 1440

3-element Vector{Float64}:
 1440.0
   43.72030729672949
    0.4048941707691826

In [45]:
using DataFrames
table = DataFrame("Method"=>method_names,"Speedup" => speedup, "Runtime" => minutes);

print(table)

3×3 DataFrame
 Row │ Method  Speedup      Runtime     
     │ String  Float64      Float64     
─────┼──────────────────────────────────
   1 │ Naive   1.0          1440.0
   2 │ Horner  0.0303613      43.7203
   3 │ Macro   0.000281177     0.404894

In [ ]:
median

In [29]:
table = DataFrame("Method"=>["Naive","Horner"],"Speedup" => [1.0, speedup1]);

10000-element Vector{Float64}:
 0.093
 0.087
 0.072
 0.072
 0.071
 0.071
 0.071
 0.066
 0.071
 0.071
 0.071
 0.068
 0.069
 ⋮
 0.05
 0.051
 0.051
 0.05
 0.06
 0.05
 0.05
 0.06
 0.061
 0.05
 0.05
 0.059